# Linear Regression in Gluon
### Generating Data Sets

In [1]:
from mxnet import autograd, nd

num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

### Reading Data

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10
# Combine the features and labels of the training data
dataset = gdata.ArrayDataset(features, labels)
# Randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

# Read a batch to see how it works
for X, y in data_iter:
    print(X, y)
    break


[[ 0.57423294  0.05569246]
 [ 0.425275   -0.37855875]
 [-0.8622762  -0.22448541]
 [ 1.3944823   0.56141484]
 [-0.29296082 -0.30558026]
 [-0.7698555   1.1850222 ]
 [ 1.7984186   0.80962783]
 [ 0.68650365  2.0535662 ]
 [ 1.7466464   0.4392858 ]
 [ 2.0691068  -0.13823958]]
<NDArray 10x2 @cpu(0)> 
[ 5.168744   6.3314323  3.25092    5.090474   4.6608973 -1.3880213
  5.0375466 -1.4055421  6.1877666  8.806868 ]
<NDArray 10 @cpu(0)>


### Define the Model

In [3]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

### Initialize Model Parameters

- Initialize weight parameter by a normal distribution with a mean of 0 and standard deviation of 0.01.
- The bias parameter is initialized to zero by default.

In [4]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

### Define the Loss Function

In [5]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()  # The squared loss is also known as the L2 norm loss

### Define the Optimization Algorithm

In [6]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### Training

In [7]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.034961
epoch 2, loss: 0.000124
epoch 3, loss: 0.000049


### Evaluate 

In [8]:
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)

Error in estimating w 
[[ 0.00054348 -0.00042605]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[0.00072241]
<NDArray 1 @cpu(0)>
